In [1]:
import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import copy
from typing import Literal
import argparse
import pandas as pd
import numpy as np
import os

import torch
from torch import nn
from pathlib import Path

from torch_uncertainty.utils import TUTrainer
from torch import nn

from torch_uncertainty.datamodules import MNISTDataModule
from torch_uncertainty.models.lenet import lenet
from torch_uncertainty.models import mc_dropout
from torch_uncertainty.models.mlp import bayesian_mlp
from torch_uncertainty.optim_recipes import optim_cifar10_resnet18
from torch_uncertainty.routines import ClassificationRoutine


from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm


In [2]:
### load the dataset

results_dict = {}

labeling_functions=[
    "guo_los",
    "guo_readmission",
    "guo_icu",
    "new_hypertension",
    "new_hyperlipidemia",
    "new_pancan",
    "new_celiac",
    "new_lupus",
    "new_acutemi",
    "lab_thrombocytopenia",
    "lab_hyperkalemia",
    "lab_hyponatremia",
    "lab_anemia",
    "lab_hypoglycemia" # will OOM at 200G on `gpu` partition
]


def optim_recipe(model, lr_mult: float = 1.0):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05 * lr_mult)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    return {"optimizer": optimizer, "lr_scheduler": scheduler}

max_epochs = 50
batch_size = 64

# for i in tqdm(range(len(labeling_functions))):
for i in tqdm(range(10, len(labeling_functions))):
    task = labeling_functions[i]
    folder_path = f'single_task_data/{task}'

    train_x_name = os.path.join(folder_path, 'X_train.csv')
    train_y_name = os.path.join(folder_path, 'y_train.csv')
    val_x_name = os.path.join(folder_path, 'X_val.csv')
    val_y_name = os.path.join(folder_path, 'y_val.csv')
    test_x_name = os.path.join(folder_path, 'X_test.csv')
    test_y_name = os.path.join(folder_path, 'y_test.csv')

    X_train = pd.read_csv(train_x_name).to_numpy()
    y_train = pd.read_csv(train_y_name).to_numpy().reshape(-1)
    X_val = pd.read_csv(val_x_name).to_numpy()
    y_val = pd.read_csv(val_y_name).to_numpy().reshape(-1)
    X_test = pd.read_csv(test_x_name).to_numpy()
    y_test = pd.read_csv(test_y_name).to_numpy().reshape(-1)

    # Create class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    X_train = torch.tensor(X_train).float()
    X_val = torch.tensor(X_val).float()
    X_test = torch.tensor(X_test).float()
    y_train = torch.tensor(y_train).long()
    y_val = torch.tensor(y_val).long()
    y_test = torch.tensor(y_test).long()

    # Create TensorDatasets
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoaders
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
    val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)
    test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

    model = bayesian_mlp(in_features = 768, num_outputs = 2, hidden_dims = [128])

    loss = ELBOLoss(
        model=model,
        inner_loss=nn.CrossEntropyLoss(weight=class_weights),
        kl_weight=1 / 500,
        num_samples=3,
    )

    routine = ClassificationRoutine(
        model=model,
        num_classes=2,
        loss=loss,
        optim_recipe=optim_recipe(model, 2),
        is_ensemble=True
    )

    trainer = Trainer(accelerator="gpu", enable_progress_bar=True, max_epochs=max_epochs)

    trainer.fit(routine, train_dataloaders=train_dl, val_dataloaders=val_dl)

    perf = trainer.test(model=routine, dataloaders=test_dl)

    results_dict[task] = perf

    del trainer, routine, model
    del train_dl, val_dl, test_dl

  0%|          | 0/4 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | StochasticModel  | 197 K  | train
1 | loss                | ELBOLoss         | 197 K  | train
2 | format_batch_fn     | Identity         | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entropy     | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.28129124641418457    │
│       test/cal/aECE       │    0.2812914252281189     │
│       test/cls/Acc        │    0.9687176942825317     │
│      test/cls/Brier       │    0.2173401415348053     │
│       test/cls/NLL        │    0.3916487693786621     │
│     test/cls/entropy      │    0.6067569255828857     │
│   test/ens_Disagreement   │    0.31763625144958496    │
│     test/ens_Entropy      │    0.5020381212234497     │
│        test/ens_MI        │    0.10471842437982559    │
│       test/sc/AUGRC       │   0.004975947085767984    │
│       test/sc/AURC        │   0.008033107966184616    │
│    test/sc/CovAt5Risk     │            1.0            │
│    test/sc/RiskAt80Cov    │   0.009408986195921898    │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 4/4 [5:29:23<00:00, 4940.75s/it]  


In [3]:
import json
with open('results_single_bnn_2.json', 'w') as f:
    json.dump(results_dict, f)

In [4]:
results_dict.keys()

dict_keys(['lab_hyperkalemia', 'lab_hyponatremia', 'lab_anemia', 'lab_hypoglycemia'])

In [5]:
labeling_functions[10:]

['lab_hyperkalemia', 'lab_hyponatremia', 'lab_anemia', 'lab_hypoglycemia']